In [ ]:
import pathlib
import pandas
import matplotlib
import matplotlib.pyplot
import numpy

File paths

In [ ]:
base_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data\roughness')
l_file_1 = pathlib.Path(r'BH35_0709_A.txt')
l_file_2 = pathlib.Path(r'BH35_0709_B.txt')
l_file_3 = pathlib.Path(r'BH35_0709_C.txt')
l_file_4 = pathlib.Path(r'BH35_0709_D.txt')
r_file_1 = pathlib.Path(r'originalZoValuesZo.txt')
r_file_2 = pathlib.Path(r'Standardised 1_to_65535Zo.txtZo.txt')
r_file_3 = pathlib.Path(r'StandardisedZo_scaledMin_to_Max.txt')

Read in Lidar and combine

In [ ]:
lidar_1=pandas.read_csv(base_path/l_file_1, names=['x','y','z','val'], sep=' ').sort_values(by=['x', 'y'])
lidar_2=pandas.read_csv(base_path/l_file_2, names=['x','y','z','val'], sep=' ').sort_values(by=['x', 'y'])
lidar_3=pandas.read_csv(base_path/l_file_3, names=['x','y','z','val'], sep=' ').sort_values(by=['x', 'y'])
lidar_4=pandas.read_csv(base_path/l_file_4, names=['x','y','z','val'], sep=' ').sort_values(by=['x', 'y'])

lidar = pandas.concat([lidar_1, lidar_2, lidar_3, lidar_4], ignore_index=True)

Read in roughness

In [ ]:
original=pandas.read_csv(base_path/r_file_1, names=['x','y','z','val'], sep=',').sort_values(by=['x', 'y'])
standard_full=pandas.read_csv(base_path/r_file_2, names=['x','y','z','val'], sep=',').sort_values(by=['x', 'y'])
standard_diff=pandas.read_csv(base_path/r_file_3, names=['x','y','z','val'], sep=',').sort_values(by=['x', 'y'])

# Examine Graham's results

### Histograms

LiDAR

In [ ]:
f = matplotlib.pyplot.figure(figsize=(15, 15))

ax1 = f.add_subplot(221) 
ax = lidar_1.hist(column = 'val', bins=100, ax=ax1, label="lidar A")
ax1.set_title('LiDAR A')

ax2 = f.add_subplot(222) 
ax = lidar_2.hist(column = 'val', bins=100, ax=ax2, label="lidar B")
ax2.set_title('LiDAR B')

ax3 = f.add_subplot(223) 
ax = lidar_3.hist(column = 'val', bins=100, ax=ax3, label="lidar C")
ax3.set_title('LiDAR C')

ax4 = f.add_subplot(224) 
ax = lidar_4.hist(column = 'val', bins=100, ax=ax4, label="lidar D")
ax4.set_title('LiDAR D')

The maximum values look like an overflow/max error. I wonder if they are associated with an integer max. 

In [ ]:
numpy.iinfo(numpy.uint16).max

### Roughness

Note that the values with a 1 in the two standardised files should be the same and should be ignored in those files and also the original file.

In [ ]:
(len(original), len(standard_full), len(standard_diff))

In [ ]:
(len(standard_full['val']!=1), len(standard_diff['val']!=1))

Plot the histograms (values of 1 to ignore - aren't shown)

In [ ]:
mask = standard_full['val']!=1

f = matplotlib.pyplot.figure(figsize=(15, 5))

ax1 = f.add_subplot(131) 
ax = original[mask].hist(column = 'val', bins=100, ax=ax1, label="original", range=(0, 0.025))
ax1.set_title('Manual roughness')

ax2 = f.add_subplot(132) 
ax = standard_full[mask].hist(column = 'val', bins=100, ax=ax2, label="standard_full", range=(0, 0.025))
ax2.set_title('Standardised full range')

ax3 = f.add_subplot(133) 
ax = standard_diff[mask].hist(column = 'val', bins=100, ax=ax3, label="standard_diff", range=(0, 0.025))
ax3.set_title('Standardised min vs max')

### 2D scatter plots

LiDAR

In [ ]:
n_thining = 100

f = matplotlib.pyplot.figure(figsize=(15, 15))
ax_l1 = lidar.iloc[::n_thining, :].plot.scatter(x='x', y='y', c='val', colormap = 'viridis')#, s='z')

Roughness

In [ ]:
n_thining = 10
mask = standard_full['val']!=1

f = matplotlib.pyplot.figure(figsize=(15, 5))

ax1 = f.add_subplot(131) 
ax_r1 = original[mask].iloc[::n_thining, :].plot.scatter(x='x', y='y', c='val', colormap = 'viridis', ax=ax1, 
                                                      title="original", vmin=0, vmax=0.025)

ax2 = f.add_subplot(132) 
ax_r2 = standard_full[mask].iloc[::n_thining, :].plot.scatter(x='x', y='y', c='val', ax=ax2, 
                                                                                 colormap = 'viridis', title="standard_full", 
                                                                                 vmin=0, vmax=0.025)

ax3 = f.add_subplot(133) 
ax_r3 = standard_diff[mask].iloc[::n_thining, :].plot.scatter(x='x', y='y', c='val', ax=ax3, 
                                                                                 colormap = 'viridis', title="standar_diff", 
                                                                                 vmin=0, vmax=0.025)

The difference between the two standardisation approaches and the manual approach

In [ ]:
n_thining = 10
mask = standard_full['val']!=1
difference = original[mask].copy()

f = matplotlib.pyplot.figure(figsize=(15, 5))

difference.loc[:, 'val'] = original[mask]['val']-standard_full[mask]['val']
ax1 = f.add_subplot(121) 
ax_r1 = difference.iloc[::n_thining, :].plot.scatter(x='x', y='y', c='val', colormap = 'viridis', ax=ax1, 
                                                      title="original", vmin=-0.01, vmax=0.01)

difference.loc[:, 'val'] = original[mask]['val']-standard_diff[mask]['val']
ax2 = f.add_subplot(122) 
ax_r2 = difference.iloc[::n_thining, :].plot.scatter(x='x', y='y', c='val', ax=ax2, 
                                                                                 colormap = 'viridis', title="standard_full", 
                                                                                 vmin=-0.01, vmax=0.01)

Check for spatial distribution in errors - only keep the ones that a greater/less than +/-0.05

In [ ]:
mask = standard_full['val']!=1
difference = original[mask].copy()

f = matplotlib.pyplot.figure(figsize=(15, 5))

difference.loc[:, 'val'] = original[mask]['val']-standard_full[mask]['val']
ax1 = f.add_subplot(121) 
ax_r1 = difference[(difference['val']>0.01) | (difference['val']<-0.01)].plot.scatter(x='x', y='y', c='val', colormap = 'viridis', 
                                                                                      ax=ax1, vmin=-0.015, vmax=0.015, 
                                                                                      title="error w/ full standardisation")

difference.loc[:, 'val'] = original[mask]['val']-standard_diff[mask]['val']
ax2 = f.add_subplot(122) 
ax_r2 = difference[(difference['val']>0.01) | (difference['val']<-0.01)].plot.scatter(x='x', y='y', c='val', colormap = 'viridis',
                                                                                      ax=ax2, vmin=-0.015, vmax=0.015,
                                                                                      title="error w/ diff standardisation", 
                                                                                 )

# Explore LiDAR data
### Spatial distribution of  max density values
Look at the total number of lidar values, and also look at the number that are the maximum value

In [ ]:
len(lidar_1)

In [ ]:
(lidar_1['val']==numpy.iinfo(numpy.uint16).max).sum()

### Plot spatial distribution of max values

In [ ]:
n_thining = 1

f = matplotlib.pyplot.figure(figsize=(15, 15))

ax1 = f.add_subplot(221) 
ax_r1 = lidar_1[lidar_1['val']==numpy.iinfo(numpy.uint16).max].iloc[::n_thining, :].plot.scatter(x='x', y='y', 
                                                                                                 colormap = 'viridis', ax=ax1, 
                                                                                                 title="LiDAR A")

ax2 = f.add_subplot(222) 
ax_r2 = lidar_2[lidar_2['val']==numpy.iinfo(numpy.uint16).max].iloc[::n_thining, :].plot.scatter(x='x', y='y', 
                                                                                                 colormap = 'viridis', ax=ax2, 
                                                                                                 title="LiDAR B")

ax3 = f.add_subplot(223) 
ax_r3 = lidar_3[lidar_3['val']==numpy.iinfo(numpy.uint16).max].iloc[::n_thining, :].plot.scatter(x='x', y='y', 
                                                                                                 colormap = 'viridis', ax=ax3,
                                                                                                 title="LiDAR C")

ax4 = f.add_subplot(224) 
ax_r4 = lidar_4[lidar_4['val']==numpy.iinfo(numpy.uint16).max].iloc[::n_thining, :].plot.scatter(x='x', y='y', 
                                                                                                 colormap = 'viridis', ax=ax4,
                                                                                                 title="LiDAR D")

### Plot the spatial distribution of non-maxed values

In [ ]:
n_thining = 10

f = matplotlib.pyplot.figure(figsize=(15, 15))

ax1 = f.add_subplot(221) 
ax_r1 = lidar_1[lidar_1['val']<numpy.iinfo(numpy.uint16).max].iloc[::n_thining, :].plot.scatter(x='x', y='y', c='val', 
                                                                                                 colormap = 'viridis', ax=ax1, 
                                                                                                 title="LiDAR A")

ax2 = f.add_subplot(222) 
ax_r2 = lidar_2[lidar_2['val']<numpy.iinfo(numpy.uint16).max].iloc[::n_thining, :].plot.scatter(x='x', y='y', c='val', 
                                                                                                 colormap = 'viridis', ax=ax2, 
                                                                                                 title="LiDAR B")

ax3 = f.add_subplot(223) 
ax_r3 = lidar_3[lidar_3['val']<numpy.iinfo(numpy.uint16).max].iloc[::n_thining, :].plot.scatter(x='x', y='y', c='val', 
                                                                                                 colormap = 'viridis', ax=ax3,
                                                                                                 title="LiDAR C")

ax4 = f.add_subplot(224) 
ax_r4 = lidar_4[lidar_4['val']<numpy.iinfo(numpy.uint16).max].iloc[::n_thining, :].plot.scatter(x='x', y='y', c='val', 
                                                                                                 colormap = 'viridis', ax=ax4,
                                                                                                 title="LiDAR D")